In [199]:
import pandas as pd
import numpy as np

In [200]:
# read data
biomarker_clean = pd.read_csv('../data/biomarker_clean.csv')

In [201]:
# see column names
biomarker_clean.columns

Index(['Unnamed: 0', 'group', 'ados', 'CHIP', 'CEBPB', 'NSE', 'PIAS4',
       'IL-10 Ra', 'STAT3', 'IRF1',
       ...
       'UB2G2', 'Transgelin-2', 'ATPO', 'Corticotropin-lipotropin', 'QORL1',
       'PEDF', 'CATF', 'FTCD', 'UBP25', 'PLXB2'],
      dtype='object', length=1320)

In [202]:
# drop the unnamed column and ados
biomarker_clean = biomarker_clean.drop(['Unnamed: 0'], axis=1)

ADOS diagnostic algo- rithms consisting of two behavioral domains: Social Affect (SA) and Restricted and Repetitive Behaviors (RRB) were used to determine an ADOS total score, which provides a continuous measure of overall ASD symptom severity.

In [203]:
biomarker_clean[biomarker_clean['ados'].isna()]['group'].value_counts()

group
TD    78
Name: count, dtype: int64

In [204]:
biomarker_clean[biomarker_clean['ados'].notna()]['group'].value_counts()

group
ASD    76
Name: count, dtype: int64

# Binomial classification

In [205]:
biomarker_clean = biomarker_clean.drop(['ados'], axis=1)


In [206]:
biomarker_clean['group'] = biomarker_clean['group'].apply(lambda x: 1 if x == 'ASD' else 0).astype(int)

In [207]:
X = biomarker_clean.loc[:, 'CHIP':].to_numpy()
y = biomarker_clean.loc[:, 'group'].to_numpy()
X

array([[ 0.33500909,  0.52030255, -0.55429753, ...,  3.        ,
        -0.47723488, -1.23419401],
       [-0.07145438,  1.00627423,  3.        , ...,  0.78371039,
         0.13825367,  0.0952122 ],
       [-0.4060154 , -0.53103679, -0.05922125, ..., -1.14594023,
        -0.68955127,  0.83858575],
       ...,
       [ 0.03237458,  1.32008149, -0.24124172, ..., -0.10017329,
         0.49931348,  0.63298266],
       [ 0.19662737,  1.5288767 ,  0.36405519, ..., -0.23166439,
         0.65427786, -0.03871574],
       [ 1.24956203,  0.47847807,  3.        , ...,  1.66966352,
        -0.58197324,  1.52234164]])

In [208]:
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

sklearn.utils.resample(*arrays, replace=True, n_samples=None, random_state=None, stratify=None)[source]
Resample arrays or sparse matrices in a consistent way.

The default strategy implements one step of the bootstrapping procedure.

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=197)

## Random Forest

In [210]:
rf_model = RandomForestClassifier(n_estimators=10,
                                  criterion='gini',
                                  max_depth=15,
                                  min_samples_split=5,
                                  bootstrap=True,
                                  random_state=197)
rf_model.fit(X_train, y_train)
rf_feature_importance = rf_model.feature_importances_

In [211]:
rf_top_feature_importance_idx = np.argsort(rf_feature_importance)[-20:][::-1]

In [219]:
rf_top_feature_importance_idx

array([1192,  988, 1144,  241, 1115,  292,  643,  201,   52,  193,  366,
        845,  290, 1252,    6,  387,  964,  429,  926,  701])

In [220]:
rf_biomarker_clean = biomarker_clean.columns[rf_top_feature_importance_idx]

## Select top 20 proteins

In [213]:
X_rf_top_20 = X_train[:, rf_top_feature_importance_idx]
y_rf_top_20 = y_train
lg_feature_importance = np.zeros(X_rf_top_20.shape[1]) # size = number of features

In [214]:
for i in range(50):
  X_bootstrap, y_bootstrap = resample(X_rf_top_20, y_rf_top_20, random_state=i)

  lg_model = LogisticRegression(solver='liblinear', C=1, penalty='l1', random_state=i).fit(X_bootstrap, y_bootstrap)

  lg_feature_importance += (lg_model.coef_ != 0).sum(axis=0) # sum over rows

lg_feature_importance /= 50 # calculate the average of non-zero features


In [215]:
unique_value, counts = np.unique(lg_feature_importance, return_counts=True)

In [216]:
print(f'Average Non-zero coefficient frequency of appearance in bootsrap: {unique_value}')
print(f'Counts of each average non-zero coefficient frequency: {counts}')

Average Non-zero coefficient frequency of appearance in bootsrap: [0.52 0.58 0.62 0.64 0.66 0.68 0.74 0.76 0.82 0.88 0.94 0.98 1.  ]
Counts of each average non-zero coefficient frequency: [1 1 2 1 2 3 1 3 1 1 1 1 2]


In [217]:
feature_importance_top_9_idx = np.argsort(lg_feature_importance)[-9:][::-1] # select top 20 feature indices

print(f'Top 20 Non-zero coefficient frequency INDEX of appearance in boot strap: {feature_importance_top_9_idx}')
print(f'Top 20 Non-zero coefficient frequency VALUE of appearance in boot strap: {lg_feature_importance[feature_importance_top_9_idx]}')

Top 20 Non-zero coefficient frequency INDEX of appearance in boot strap: [17  1 11  6  8  2 15 12 18]
Top 20 Non-zero coefficient frequency VALUE of appearance in boot strap: [1.   1.   0.98 0.94 0.88 0.82 0.76 0.76 0.76]


## 5 Proteins that have the most average weights by 100-bootstrap

In [226]:
lg_biomarker_colnames = rf_biomarker_clean[feature_importance_top_9_idx]
lg_biomarker_colnames

Index(['CD39', 'DBNL', 'Collectin Kidney 1', 'OCAD1', 'BCL6', 'Prothrombin',
       'Flt3 ligand', 'ERK-1', 'ITI heavy chain H4'],
      dtype='object')

In [225]:
X_final_training = biomarker_clean.loc[:, lg_biomarker_colnames]
X_final_training

,CD39,DBNL,Collectin Kidney 1,OCAD1,BCL6,Prothrombin,Flt3 ligand,ERK-1,ITI heavy chain H4
0,0.558961,0.503559,-1.043312,-0.148950,0.228153,-0.340711,0.050875,-0.195837,0.389372
1,0.102769,-0.296588,-0.490555,-0.625926,0.252370,-0.414998,0.019506,0.889466,-1.212895
2,-0.618390,1.377972,0.571774,-0.417502,-0.268227,1.247360,-0.323503,-0.180621,0.255682
3,-0.329925,-0.497788,0.129357,0.341663,-0.005226,-0.973634,-0.537784,2.235239,-0.058901
4,-0.241031,0.070020,0.140418,0.590670,-0.079062,-0.891510,-0.355734,1.388327,-0.393853
...,...,...,...,...,...,...,...,...,...
149,-0.649712,0.103900,1.292436,-0.148528,-0.046210,0.543231,1.962896,0.091265,0.161999
150,-0.152028,0.426534,0.079957,0.547445,0.122472,0.059372,0.133722,1.379568,-0.142699
151,0.794906,-0.109869,-0.117592,0.666404,1.413774,-1.742080,0.650453,-0.665855,0.356584
152,0.776736,-0.517024,-1.989805,1.012334,0.822150,-0.816188,0.665157,1.484037,-0.121631
